In [0]:
checkpoint= "abfss://checkpoint@adlsexamplebiju.dfs.core.windows.net"  

In [0]:
from pyspark.sql import functions as F

def handle_null_values(df):
    print("****Handling NULL Values****")
    print("Replacing NULL values on String Columns with 'Unknown'")
    df = df.fillna('Unknown')
    print('Success!! ')

    print("Replacing NULL values on Numeric Columns with 0")
    df = df.fillna(0)
    print('Success!! ')
    
    return df

In [0]:
def remove_duplicates(df):
    print("****Removing Duplicates****")
    df = df.dropDuplicates()
    print('Duplicates Removed!! ')
    return df

In [0]:
def read_BronzeTrafficTable():
    print("Reading the Bronze Table Data")
    df_bronzeTraffic = (spark.readStream
                    .table("my_catalog.bronze.raw_traffic")
                    )
    print(f'Reading MY_catalog.bronze.raw_traffic Success!')
    return df_bronzeTraffic

In [0]:
read_BronzeTrafficTable()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp

In [0]:
def process_traffics_data(df):
    print("****Processing Roads Data****")

    # Create Electric Vehicles Count Column
    print('Creating Electric Vehicles Count Column')
    df = df.withColumn('Electric_Vehicles_Count',
                        F.col('EV_Car') + F.col('EV_Bike'))
    print('Success!! ')

    
    
    # Create All Motor Vehicles Count Column
    print('Creating All Motor Vehicles Count Column')
    df = df.withColumn('Motor_Vehicles_Count',
                        F.col('Electric_Vehicles_Count') +
                        F.col('Two_wheeled_motor_vehicles') +
                        F.col('Cars_and_taxis') +
                        F.col('Buses_and_coaches') +
                        F.col('LGV_Type') +
                        F.col('HGV_Type'))
    print('Success!! ')

    # Add a Quality Check Status Column
    df = df.withColumn("Quality_Check", F.lit("Passed"))
    
    # Add a Transformed Time Column
    df = df.withColumn("Transformed_Time", current_timestamp())

    # Fix "Count_date" Date Format
    df = df.withColumn(
        "Count_date",
        F.to_date(F.to_timestamp(F.col("Count_date"), "M/d/yyyy H:mm"))  # Flexible format for 1 or 2 digits
    )

    print("****Processing Completed****")
    return df



In [0]:
df = read_BronzeTrafficTable()
process_traffics_data(df)

In [0]:
def read_BronzeRoadsTable():
    print("Reading the Bronze Roads Table Data")
    df_bronzeRoads = (spark.readStream
                    .table("my_catalog.bronze.raw_roads")
                    )
    print(f'Reading my_catalog.bronze.raw_roads Success!')
    return df_bronzeRoads


In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category_ID') == '3', 'Class A Trunk Road')
                  .when(col('Road_Category_ID') == '1', 'Class A Trunk Motor')
                   .when(col('Road_Category_ID') == '4','Class A Principal road')
                    .when(col('Road_Category_ID') == '2','Class A Principal Motorway')
                    .when(col('Road_Category_ID') == '5','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat

In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type


In [0]:
def write_Roads_SilverTable(StreamingDF):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable("my_catalog.silver.silver_roads"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing my_catalog.silver.silver_roads Success!')


In [0]:
def write_Traffic_SilverTable(StreamingDF):
    print("Writing the silver_traffic Data") 

    write_StreamSilver = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverTrafficLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable("my_catalog.silver.silver_traffic"))
    
    write_StreamSilver.awaitTermination()
    print(f'Writing silver_traffic Success!')


In [0]:
df_bronzeTraffic = read_BronzeTrafficTable()
df_dup = remove_duplicates(df_bronzeTraffic)
df_null = handle_null_values(df_dup)
df_process=process_traffics_data(df_null)

df_bronzeRoads = read_BronzeRoadsTable()
df_Roadsdup = remove_duplicates(df_bronzeRoads)
df_Roadsnull = handle_null_values(df_Roadsdup)
df_roadCat = road_Category(df_Roadsnull)
df_type = road_Type(df_roadCat)
write_Roads_SilverTable(df_type)
write_Traffic_SilverTable(df_process)